In [22]:
import os
import pandas as pd
import numpy as np
import re
from shutil import copyfile

In [ ]:
imgs_modes=['t1dual','t2spir','inphase','outphase','dicom_anon','ground','source']
src_dir=''
pair_keyword=[]
dst_grnd_path=''
dst_img_path=''
grnd_sub_repl_tple_dict={}
src_sub_repl_tple_dict={}

In [ ]:
#Renaming pipeline

file_dict=gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1)

file_tuple_lst=gen_file_tpl_set_lists(file_dict,pair_keyword)

for vals in file_tuple_lst:
    
    rename_file_set_for_segmentation(vals,
                                         grnd_sub_repl_tple_dict,
                                        src_sub_repl_tple_dict,
                                         dst_grnd_path,
                                        dst_img_path)

In [21]:
def gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1):
    """The purpose of this method is to evaluate an estimation """
    file_dict={}
    for subdir,root,files in os.walk(src_dir):
        print(subdir)

        if len(files)!=0:
            gt_img_name=[]
            gt_img_name=[''.join(x) for x in imgs_modes if x in subdir.lower()]
            #If two categories 
            if len(gt_img_name)>=key_word_sp:

                tmp_file_lst=[os.path.join(subdir,file) for file in files]
                seperator = '_'
                seperator =seperator.join(gt_img_name)
                file_dict[seperator]=sorted(tmp_file_lst)
    #Sett original name for analysis and munging
    return file_dict

File name 1 T1Dual inphase:
1.	Pat_10_t1dual_ground_slice_no_10_512x512.png
2.	Pat_10_t1dual_inphase_slice_no_10_512x512_grey_lvl_256.png
File name 2 T1Dual  outphase:
1.	Pat_10_t1dual_ground_slice_no_10_512x512.png
2.	Pat_10_t1dual_outphase_slice_no_10_512x512_grey_lvl_256.png
File name 3 T2SPIR
1.	Pat_10_t2spir_slice_no_10_512x512_grey_lvl_256.png
2.	Pat_10_t2spir_ground_slice_no_10_512x512.png


In [24]:
def gen_file_tpl_set_lists(file_dict:dict,pair_keyword: list)->list:
    """The purpose of this method is to generate keyword base file tuple sets"""
    #final list to return list
    rtrn_lst=[]
    #Creating list of keys to iterate through 
    file_sets=list(file_dict.keys()) 
    
    for key in pair_keyword:
        #Get main image set initially b yimage type i.e. t1dual
        img_type_lst=[x for x in file_sets if x.lower().find(key[0])!=-1]
        #Getting the ground key
        img_grnd_key=[x for x in img_type_lst if x.lower().find(key[1])!=-1]
        #Getting non ground by removing that value from the list 
        img_keys=img_type_lst.remove(img_grnd_key[0])
        #Generating file list tuple pairs for anaylysis
        if len(img_keys)>1:
            file_pair=[zip(file_dict[img_key],file_dict[img_grnd_key]) for img_key in img_keys]
            
        else:
            file_pair=[zip(file_dict[img_keys],file_dict[img_grnd_key])]
            
        for tpl_vals in file_pair:
            rtrn_lst.append(tpl_vals)
            
    return rtrn_lst


In [25]:
def repl_sub_string(str_val:str,str_rep_dict:dict):
    """The purpose of this method is to itertivley replace substrings in a string with other values"""
    
    # Add escape characters to string keys for regex estimation
    rep = dict((re.escape(k), v) for k, v in str_rep_dict.items()) 
    #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
    pattern = re.compile("|".join(str_rep_dict.keys()))
    
    return pattern.sub(lambda m: rep[re.escape(m.group(0))],str_val)

In [26]:
def rename_file_set_for_segmentation(file_set_list_tuple:list,
                                     grnd_sub_repl_tple_dict:dict,
                                    src_sub_repl_tple_dict:dict,
                                     dst_grnd_path:str,
                                    dst_img_path:str):
    """The purpose of this method is to detail how """
    
    
    for grnd_nm_path,img_nm_path in file_set_list_tuple:
        
        grnd_nm=os.path.basename(grnd_nm_path)
        img_nm=os.path.basename(img_nm_path)
        
        strp_grnd_nm=repl_sub_string(grnd_nm,grnd_sub_repl_tple_dict)
        strp_img_nm=repl_sub_string(img_nm,src_sub_repl_tple_dict)
        
        dst_grnd_path=os.path.join(dst_grnd_path,strp_grnd_nm)
        dst_img_path=os.path.join(dst_img_path,strp_img_nm)
        
        copyfile(grnd_nm_path, dst_grnd_path)
        copyfile(img_nm_path,dst_img_path)